<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [40]:
import pandas as pd
import numpy as np

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords

import gensim.corpora as corpora
from gensim.models import CoherenceModel

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models.ldamulticore import LdaMulticore

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [38]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
nlp = spacy.load('en_core_web_lg')

In [5]:
def tokenize(doc):
    tokens = simple_preprocess(doc)
    return tokens

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [13]:
tfidf = TfidfVectorizer(stop_words='english')

sparse = tfidf.fit_transform(yelp.text)

In [15]:
dtm_tfidf = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

In [133]:
fake_review = "Terrible, horrible, disgusting. I will never forgive. I will never forget. Bad.  May the plague descend upon the orchestrators of my misfortune."

In [134]:
tokenized_review = tokenize(fake_review)

In [116]:
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm_tfidf)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=2, p=2, radius=1.0)

In [135]:
query = tfidf.transform(tokenized_review)

results = nn.kneighbors(query.todense())

In [136]:
results

(array([[0.78770877, 0.8435033 ],
        [0.94335743, 1.06494876],
        [1.08638262, 1.13996368],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.05319219, 1.2822979 ],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.12865996, 1.12893219],
        [0.76723817, 0.93695451],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.26650621, 1.30607639],
        [1.28105719, 1.34888228],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.        , 1.        ],
        [1.22056787, 1.41421356]]), array([[2627, 5298],
        [7235, 1922],
        [9900, 1759],
        [4837, 7181],
        [4837, 7181],
        [3541, 5725],
        [4837, 7181],
        [4837, 7181],
        [1298, 2867],
        [4599, 2518],
        [4837, 7181],
        [4837, 7181],
        [2285, 9500],
        [2888, 1316],
        [48

In [191]:
for i in results[1][:5]:
    print(yelp.text[i])

2627    Terrible terrible terrible went to get a "hot ...
5298    Terrible.. absolutely terrible. Dr. Hinds is e...
Name: text, dtype: object
7235    The service here is horrible. The food wasn't ...
1922    Horrible customer service and management unwil...
Name: text, dtype: object
9900    How they make a person wait  2 hours for a 2 m...
1759    For some reason everyone gets sick after a mea...
Name: text, dtype: object
4837    Beware, I went here when this location first o...
7181    I only bother to review businesses when I thin...
Name: text, dtype: object
4837    Beware, I went here when this location first o...
7181    I only bother to review businesses when I thin...
Name: text, dtype: object


```
the results are primarily negative reviews, as expected
```

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [142]:
from sklearn.linear_model import SGDClassifier

In [61]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [41]:
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    return tokens

def clean_text(text):
    return text.strip().lower()

bow_vector = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = CountVectorizer(tokenizer=spacy_tokenizer)

class predictors(TransformerMixin):
    def transform(self,X,**transform_params):
        return [clean_text(text) for text in X]
    def fit(self,X,y=None,**fit_params):
        return self
    def get_params(self,deep=True):
        return {}

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

classifier = LogisticRegression(solver='lbfgs', multi_class='auto',max_iter=1000)

pipeline = Pipeline([('cleaner', predictors()),
                ('vectorizer', bow_vector),
                ('clf', classifier)])

In [144]:
vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()

pipeline = Pipeline([('vect', vect), ('clf', sgdc)])

In [145]:
pipeline.fit(yelp['text'], yelp['stars'])

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [148]:
pipeline.predict([fake_review])

array([1])

In [152]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__max_iter':(20, 10, 100)
}

In [157]:
grid_search = GridSearchCV(pipeline, parameters, cv=6, n_jobs=-1, verbose=1)

In [158]:
grid_search.fit(yelp.text, yelp.stars)

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   19.7s finished
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'clf__max_iter': (20, 10, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [159]:
grid_search.best_params_

{'clf__max_iter': 20,
 'vect__max_df': 1.0,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

Learn the vocubalary of the yelp data:

In [161]:
list_of_reviews = yelp.text.to_list()

In [77]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [162]:
tokenized_list = [simple_preprocess(doc) for doc in list_of_reviews]

In [163]:
for i in tokenized_list:
    for j in i:
        if j in stop_words:
            i.remove(j)

In [91]:
bigram = gensim.models.Phrases(tokenized_list, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[tokenized_list], threshold=100)

In [92]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [93]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemma(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [94]:
bigrams = make_bigrams(tokenized_list)

In [164]:
# lemmatized = lemma(bigrams)

lemmatized = lemma(tokenized_list)

In [165]:
id2word = corpora.Dictionary(lemmatized)

In [166]:
id2word.filter_extremes(no_below=20, no_above=0.85)

In [167]:
corpus = [id2word.doc2bow(text) for text in lemmatized]

In [192]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=25,
                   workers=6,
                   num_topics = 4 # You can change this parameter
                  )

In [193]:
coherence_model_lda = CoherenceModel(model=lda, texts=lemmatized,
                                    dictionary=id2word, coherence='c_v')

coherence_model_lda.get_coherence()

0.2993673092496755

In [194]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.008664 -0.003365       1        1  28.514338
3      0.009272 -0.001199       2        1  28.127855
0      0.001002 -0.004091       3        1  22.150257
1     -0.001610  0.008655       4        1  21.207546, topic_info=    Category          Freq        Term         Total  loglift  logprob
0    Default  32158.000000      -PRON-  32158.000000  30.0000  30.0000
33   Default  26046.000000         the  26046.000000  29.0000  29.0000
4    Default  18316.000000          be  18316.000000  28.0000  28.0000
44   Default   6289.000000        good   6289.000000  27.0000  27.0000
134  Default   5316.000000       place   5316.000000  26.0000  26.0000
208  Default   4243.000000       great   4243.000000  25.0000  25.0000
14   Default   6316.000000        have   6316.000000  24.0000  24.0000
197  Default   3005.000000        back   3005.000000  23.0000  23.0000
88   Default   3186.000000        make   3186.000000  22.0000  22.0000
362  Default   3493.000000       order   3493.000000  21.0000  21.0000
96   Default   3648.000000         one   3648.000000  20.0000  20.0000
13   Default   6225.000000         get   6225.000000  19.0000  19.0000
348  Default   1642.000000        find   1642.000000  18.0000  18.0000
76   Default   5704.000000          go   5704.000000  17.0000  17.0000
343  Default    905.000000        dish    905.000000  16.0000  16.0000
836  Default   1094.000000         use   1094.000000  15.0000  15.0000
34   Default   4334.000000        time   4334.000000  14.0000  14.0000
184  Default   1344.000000        menu   1344.000000  13.0000  13.0000
118  Default   2647.000000        well   2647.000000  12.0000  12.0000
297  Default   1288.000000       thing   1288.000000  11.0000  11.0000
186  Default   2708.000000      really   2708.000000  10.0000  10.0000
17   Default   3912.000000        like   3912.000000   9.0000   9.0000
396  Default   1565.000000         ask   1565.000000   8.0000   8.0000
332  Default    868.000000      cheese    868.000000   7.0000   7.0000
40   Default   3996.000000        come   3996.000000   6.0000   6.0000
187  Default   1407.000000   recommend   1407.000000   5.0000   5.0000
462  Default    591.000000        roll    591.000000   4.0000   4.0000
138  Default   1351.000000       there   1351.000000   3.0000   3.0000
38   Default   1048.000000        with   1048.000000   2.0000   2.0000
286  Default   1115.000000       other   1115.000000   1.0000   1.0000
..       ...           ...         ...           ...      ...      ...
14    Topic4   1536.370117        have   6316.891602   0.1370  -4.4687
822   Topic4    346.745850         new   1213.367188   0.2983  -5.9573
138   Topic4    377.271118       there   1351.506592   0.2748  -5.8729
161   Topic4    493.795776       price   1839.448486   0.2357  -5.6038
263   Topic4    405.148712  experience   1480.738892   0.2548  -5.8016
11    Topic4    404.798248       could   1490.035278   0.2476  -5.8025
3     Topic4    591.139893        also   2304.161133   0.1904  -5.4238
33    Topic4   4829.433105         the  26046.742188  -0.1344  -3.3234
49    Topic4    911.589294     service   3839.165039   0.1130  -4.9907
40    Topic4    925.975952        come   3996.614258   0.0885  -4.9750
187   Topic4    381.410126   recommend   1407.983032   0.2448  -5.8620
37    Topic4    452.677979        wait   1736.952393   0.2061  -5.6907
4     Topic4   3201.159912          be  18316.896484  -0.1935  -3.7346
17    Topic4    886.369629        like   3912.314941   0.0661  -5.0188
307   Topic4    651.456421         and   2726.291016   0.1193  -5.3267
13    Topic4   1221.378540         get   6225.294922  -0.0778  -4.6982
76    Topic4   1086.152222          go   5704.487305  -0.1078  -4.8155
150   Topic4    942.031250        food   5113.013672  -0.1407  -4.9579
186   Topic4    597.571777      really   2708.294922   0.0396  -5.4130
208   Topic4

```
In the LDAvis whitepaper (ref: https://www.aclweb.org/anthology/W14-3110), it is stated
that topic inferred by LDA are not always easily interpretable to humans. Unfortunately such is
the case here, as I am not able to make any definitive conclusions on the basis of my topic model. 
It would appear the topics contain positive descriptors, indicating that positive reviews are 
in the majority in the dataset. Food and service are also referenced often meaning those are often 
discussed in the processed reviews. 
```

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)